In [ ]:
%pip install ../../

In [ ]:
import torch 
from spuco.datasets import WILDSDatasetWrapper
from spuco.datasets import GroupLabeledDatasetWrapper
from spuco.utils import set_seed
from wilds import get_dataset
import torchvision.transforms as transforms
import numpy as np

from spuco.models import model_factory 
from spuco.evaluate import Evaluator
from spuco.last_layer_retrain import DFR, DISPEL

import pickle
from spuco.invariant_train import ERM 
from torch.optim import SGD

In [ ]:
set_seed(0)
device = torch.device("cuda:7")

In [ ]:
# Load the full dataset, and download it if necessary
dataset = get_dataset(dataset="waterbirds", download=True, root_dir="/home/data")

target_resolution = (224, 224)
scale = 256.0 / 224.0
transform = transforms.Compose([
        transforms.Resize((int(target_resolution[0]*scale), int(target_resolution[1]*scale))),
        transforms.CenterCrop(target_resolution),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

# Get the training set
train_data = dataset.get_subset(
    "train",
    transform=transform
)

# Get the test set
test_data = dataset.get_subset(
    "test",
    transform=transform
)

# Get the val set
val_data = dataset.get_subset(
    "val",
    transform=transform
)


In [ ]:
trainset = WILDSDatasetWrapper(dataset=train_data, metadata_spurious_label="background", verbose=True)
testset = WILDSDatasetWrapper(dataset=test_data, metadata_spurious_label="background", verbose=True)
valset = WILDSDatasetWrapper(dataset=val_data, metadata_spurious_label="background", verbose=True)

In [ ]:
# load a trained model from checkpoint
model = model_factory("resnet50", trainset[0][0].shape, 2).to(device)
ckpt_path = 'path-to-erm-model'
state_dict = torch.load(ckpt_path)
model.load_state_dict(state_dict)

In [ ]:
group_labeled_set = GroupLabeledDatasetWrapper(dataset=valset, group_partition=valset.group_partition)

dfr = DFR(
    group_labeled_set=group_labeled_set,
    model=model,
    data_for_scaler=trainset,
    device=device,
    verbose=True,
)

dfr.train()

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    sklearn_linear_model=dfr.linear_model,
    device=device,
    verbose=False
    )
evaluator.evaluate()


In [ ]:
print(evaluator.worst_group_accuracy)
print(evaluator.average_accuracy)